In [1]:
import requests
import json
import pandas as pd
import csv

In [2]:
resurlt_num = 5 #検索件数、未入力なら10
output_filename = 'python_ja_new.csv' #.csv形式で名前を入力

In [3]:
base_url = 'https://www.googleapis.com/books/v1/volumes?'
url_max = 'maxResults='+ str(resurlt_num)
lnaguage = '' #日本語なら'ja'入力
sort = 'newest' #新しい順なら'newest'を入力。デフォルト（未入力）はrelevance

search = {
    'free' : '', #free word
    'title' : 'python',
    'publisher' : '',
    'subject' : '',
    'isbn' : '',
    'lccn' : '', #Library of Congress Control Number
    'oclc' : '', #Online Computer Library Center number
}

In [4]:
title_q = '' if search['title']=='' else 'intitle:'+search['title']
publisher_q = '' if search['publisher']== '' else 'inpublisher:'+search['publisher']
subject_q = '' if search['subject']== '' else 'subject:'+search['subject']
isbn_q = '' if search['isbn']== '' else 'isbn:'+search['isbn']
lccn_q = '' if search['lccn']== '' else 'lccn:'+search['lccn']
oclc_q = '' if search['oclc']== '' else 'oclc:'+search['oclc']
q_list = [search['free'], title_q, publisher_q, subject_q, isbn_q, lccn_q, oclc_q]
q_list_none = list(filter(None, q_list))
url_q = 'q=' + '+'.join(q_list_none) #q= と各検索項目を+でつなぐ
url_lnag = '' if lnaguage == '' else 'langRestrict='+lnaguage
url_sort = '' if sort == '' else 'orderBy='+sort
url_list = [url_q, url_max, url_lnag, url_sort]
url_list_none = list(filter(None, url_list))
url = base_url + '&'.join(url_list_none)

print(url)

https://www.googleapis.com/books/v1/volumes?q=intitle:python&maxResults=5&orderBy=newest


In [5]:
response = requests.get(url).json()
totalitems = response['totalItems']
print('件数：', totalitems)

件数： 401


In [9]:
url_list

['q=intitle:python', 'maxResults=5', '', 'orderBy=newest']

In [6]:
num=resurlt_num
book_df = pd.DataFrame(index=[], columns=['タイトル', '著者','出版社', '出版日', 'ページ数', '種別', '言語', '要約']) #Data Frame作成

items_list = response['items'] #items リストデータ
for i in range(num):
    items = items_list[i] #items
    info = items.get('volumeInfo')
    title = info.get('title')
    author = info.get('authors')
    publisher = info.get('publisher')
    publisheddate = info.get('publishedDate')
    pages = info.get('pageCount')
    printtype = info.get('printType')
    description = info.get('description') #要約
    language = info.get('language')
    book_df= book_df.append({'タイトル': title,
                             '著者' : author,'出版社' : publisher,'出版日':publisheddate,'ページ数':pages,
                             '種別':printtype,'言語':language,'要約':description}, ignore_index=True)
    print(book_df)
    book_df.to_csv(output_filename)
    with open(output_filename, 'a', newline='',encoding='shift-jis') as f:
        writer = csv.writer(f)
        writer.writerow(['URL', url])
        writer.writerow(['総件数', totalitems])



                          タイトル                             著者  出版社  \
0  Pythonによるあたらしいデータ分析の教科書 第2版  [寺田 学, 辻 真吾, 鈴木 たかのり, 福島 真太朗]  翔泳社   

          出版日 ページ数    種別  言語  \
0  2022-10-24  393  BOOK  ja   

                                                  要約  
0  読者の声に応えて第2版の登場! データ分析エンジニアに必要な 基本技術をしっかり習得できる ...  
                           タイトル                               著者        出版社  \
0   Pythonによるあたらしいデータ分析の教科書 第2版    [寺田 学, 辻 真吾, 鈴木 たかのり, 福島 真太朗]        翔泳社   
1  よしもとプログラミング部と学ぶPython「超」入門教室  [株式会社リーディング・エッジ社, よしもとプログラミング部]  SBクリエイティブ   

          出版日 ページ数    種別  言語  \
0  2022-10-24  393  BOOK  ja   
1  2022-09-15  275  BOOK  ja   

                                                  要約  
0  読者の声に応えて第2版の登場! データ分析エンジニアに必要な 基本技術をしっかり習得できる ...  
1  吉本プログラミング部と一緒にプログラミングを1から学ぼう！！ ※この電子書籍は固定レイアウト...  
                                      タイトル  \
0              Pythonによるあたらしいデータ分析の教科書 第2版   
1             よしもとプログラミング部と学ぶPython「超」入門教室   
2  独学コンピューターサイエンティスト Pythonで学ぶアルゴリズムとデータ構造   


In [19]:
from datetime import datetime
def confirm_aqc(data:list) -> str:
  """
  品質情報を確認し、データを文字列で返す
  input : list
  return : str
  """
  if data[1] == 0:
    return str(data[0])
  else:
    return "品質情報を確認して下さい"

stnid = "44132" # 観測所番号 - 今回は千代田区 北の丸公園

latest_time_url = "https://www.jma.go.jp/bosai/amedas/data/latest_time.txt"
latest_time_req = requests.get(latest_time_url)
latest_datetime = datetime.strptime(latest_time_req.text, "%Y-%m-%dT%H:%M:%S%z") # タイムゾーン込みで日時文字列をdatetime型へ
yyyymmdd = latest_datetime.strftime('%Y%m%d') # 年月日　- アメダスデータ取得時に必要
h3 = ("0" + str((latest_datetime.hour//3)*3))[-2:] # 3時間ごとの時間 - アメダスデータ取得時に必要
# アメダス
amedas_url = f"https://www.jma.go.jp/bosai/amedas/data/point/{stnid}/{yyyymmdd}_{h3}.json"
amedas_req = requests.get(amedas_url)
amedas_data = amedas_req.json()
latest_key = max(amedas_data) # 最新のアメダスデータが入っているkey
latest_temp = confirm_aqc(amedas_data[latest_key]["temp"]) # 最新の気温データを取得, 品質情報を確認
latest_precipitation10m = confirm_aqc(amedas_data[latest_key]["precipitation10m"]) # 最新の降水量データを取得, 品質情報を確認

print(f"現在の気温 : {latest_temp} 度")
print(f"現在の降水量(10分あたり) : {latest_precipitation10m} mm")


現在の気温 : 8.2 度
現在の降水量(10分あたり) : 0.0 mm


In [23]:
amedas_data

{'20230118210000': {'prefNumber': 44,
  'observationNumber': 132,
  'pressure': [1014.0, 0],
  'normalPressure': [1017.0, 0],
  'temp': [8.2, 0],
  'humidity': [53, 0],
  'snow': [0, 0],
  'snow1h': [0, 0],
  'snow6h': [0, 0],
  'snow12h': [0, 0],
  'snow24h': [0, 0],
  'sun10m': [0, 0],
  'sun1h': [0.0, 0],
  'precipitation10m': [0.0, 0],
  'precipitation1h': [0.0, 0],
  'precipitation3h': [0.0, 0],
  'precipitation24h': [0.5, 0],
  'windDirection': [14, 0],
  'wind': [2.6, 0],
  'maxTempTime': {'hour': 5, 'minute': 27},
  'maxTemp': [12.4, 0],
  'minTempTime': {'hour': 19, 'minute': 52},
  'minTemp': [3.8, 0],
  'gustTime': {'hour': 1, 'minute': 51},
  'gustDirection': [13, 0],
  'gust': [6.5, 0]},
 '20230118211000': {'prefNumber': 44,
  'observationNumber': 132,
  'pressure': [1014.1, 0],
  'normalPressure': [1017.1, 0],
  'temp': [8.4, 0],
  'humidity': [51, 0],
  'sun10m': [0, 0],
  'sun1h': [0.0, 0],
  'precipitation10m': [0.0, 0],
  'precipitation1h': [0.0, 0],
  'precipitation3

In [26]:
import requests
import json
import pprint

url = 'https://zipcloud.ibsnet.co.jp/api/search'

params = {'zipcode':'2330008'}

res = requests.get(url, params=params)

data = json.loads(res.text)
pprint.pprint(data)

pprint.pprint(data['results'][0]['address1'])

{'message': None,
 'results': [{'address1': '神奈川県',
              'address2': '横浜市港南区',
              'address3': '最戸',
              'kana1': 'ｶﾅｶﾞﾜｹﾝ',
              'kana2': 'ﾖｺﾊﾏｼｺｳﾅﾝｸ',
              'kana3': 'ｻｲﾄﾞ',
              'prefcode': '14',
              'zipcode': '2330008'}],
 'status': 200}
'神奈川県'


In [33]:
# APIリクエストに必要なライブラリ
import requests
# URL＋クエリパラメータ
url = 'http://weather.livedoor.com/forecast/webservice/json/v1?city=400040'
# APIリクエストを送信
tenki_data = requests.get(url)
data2 = json.loads(tenki_data.text)
pprint.pprint(data2)

JSONDecodeError: Expecting value: line 2 column 1 (char 4)

In [32]:
tenki_data 

<Response [200]>